In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceImage

import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\mxs1980\Documents\TopoTensor\notebooks


In [8]:
message_path = "AMZN_2012-06-21_34200000_57600000_message_10.csv"
orderbook_path = "AMZN_2012-06-21_34200000_57600000_orderbook_10.csv"

msg_cols = ["Time", "Type", "OrderID", "Size", "Price", "Direction"]
messages = pd.read_csv(message_path, names=msg_cols)


In [9]:
book_cols = []
for i in range(1,11):
    book_cols +=[f"Bid{i}_Price", f"Bid{i}_Vol"]

for i in range(1,11):
    book_cols +=[f"Ask{i}_Price", f"Ask{i}_Vol"]

orderbook = pd.read_csv(orderbook_path, names=book_cols)

In [10]:
assert len(messages) == len(orderbook), "Message and orderbook files misaligned"
print(f"Loaded {len(orderbook)} events for AMZN Level-10 book.")

Loaded 269748 events for AMZN Level-10 book.


In [18]:
book_t,book_t1=orderbook.iloc[0], orderbook.iloc[1]
price_cols = [c for c in book_t.index if "Price" in c]
vol_cols = [c for c in book_t.index if "Vol" in c]

In [24]:
prices = book_t[price_cols].values
vols_t = book_t[vol_cols].values
vols_t1 = book_t1[vol_cols].values

In [ ]:
delta_vols = vols_t1 - vols_t

mask = np.abs(delta_vols) > 1e-9
prices = prices[mask]
delta_vols = delta_vols[mask]


In [30]:
prices.shape

(20,)

In [17]:
n_levels = len(prices)//2
side_signs = np.array([1] * n_levels + [-1] * n_levels)[:len(prices)]
print(delta_vols.shape)

(9,)


In [11]:
def infer_order_flow(book_t, book_t1):
    price_cols = [c for c in book_t.index if "Price" in c]
    vol_cols = [c for c in book_t.index if "Vol" in c]

    prices = book_t[price_cols].values
    vols_t = book_t[vol_cols].values
    vols_t1 = book_t1[vol_cols].values

    delta_vols = vols_t1 - vols_t

    mask = np.abs(delta_vols) > 1e-9
    prices = prices[mask]
    delta_vols = delta_vols[mask]

    n_levels = len(prices)//2
    side_signs = np.array([1] * n_levels + [-1] * n_levels)[:len(prices)]
    signed_dv = delta_vols *side_signs

    return np.column_stack([prices,signed_dv])

In [12]:
from functions import build_topotensor_from_diagram

In [13]:
vr = VietorisRipsPersistence(homology_dimensions=[1])

tensors = []
metas = []

In [ ]:
for i in range(0,len(orderbook)-1):
    pc = infer_order_flow(orderbook.iloc[i], orderbook.iloc[i+1])
    

In [14]:
for i in range(0,len(orderbook)-1,50):
    pc = infer_order_flow(orderbook.iloc[i], orderbook.iloc[i+1])
    if len(pc) < 3:
        continue
    diagrams = vr.fit_transform([pc])
    pd = diagrams[0]
    if pd.shape[1]>=3:
        h1_points = pd[pd[:,2]==1][:,:2]
    else:
        h1_points = pd
    
    persists = h1_points[:,1] - h1_points[:,0]
    h1_points = h1_points[persists > 1e-9]

    tensor, meta =build_topotensor_from_diagram(h1_points)
    tensors.append(tensor)
    metas.append(meta)

ValueError: operands could not be broadcast together with shapes (9,) (8,) 